In [13]:
### Summary of steps taken:
# 1. Environment Setup: Loaded necessary libraries and datasets, created an SQLite database.
# 2. Data Extraction: Extracted data using SQL JOIN operations.
# 3. Data Transformation: Aggregated and filtered data using SQL.
# 4. View Creation: Created a SQL view for efficient future queries.
# 5. Data Analysis and Visualization: Used Python to convert data types, analyze, and visualize the data.

In [ ]:
import pandas as pd
import sqlite3

# Load datasets
amazon_sale_report = pd.read_csv('Amazon Sale Report.csv')
cloud_warehouse_comparison = pd.read_csv('Cloud Warehouse Compersion Chart.csv')
pl_march_2021 = pd.read_csv('P L March 2021.csv')
may_2022 = pd.read_csv('May-2022.csv')
sale_report = pd.read_csv('Sale Report.csv')
international_sale_report = pd.read_csv('International sale Report.csv')
expense_iigf = pd.read_csv('Expense IIGF.csv')

# Display the first few rows of each dataset to understand the structure
print("Amazon Sale Report:\n", amazon_sale_report.head(), "\n")
print("Cloud Warehouse Comparison:\n", cloud_warehouse_comparison.head(), "\n")
print("PL March 2021:\n", pl_march_2021.head(), "\n")
print("May 2022:\n", may_2022.head(), "\n")
print("Sale Report:\n", sale_report.head(), "\n")
print("International Sale Report:\n", international_sale_report.head(), "\n")
print("Expense IIGF:\n", expense_iigf.head(), "\n")

# Create a SQLite database in memory
conn = sqlite3.connect(':memory:')

# Write dataframes to SQL tables
amazon_sale_report.to_sql('amazon_sale_report', conn, index=False)
cloud_warehouse_comparison.to_sql('cloud_warehouse_comparison', conn, index=False)
pl_march_2021.to_sql('pl_march_2021', conn, index=False)
may_2022.to_sql('may_2022', conn, index=False)
sale_report.to_sql('sale_report', conn, index=False)
international_sale_report.to_sql('international_sale_report', conn, index=False)
expense_iigf.to_sql('expense_iigf', conn, index=False)

# Verify tables were created
print("Tables in the database:", pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn))


In [ ]:
# SQL query to extract and join data

query_sales = """
SELECT s.Date, s.Sales, i.Sales AS International_Sales
FROM sale_report s
JOIN international_sale_report i ON s.Date = i.Date
WHERE s.Date >= '2022-01-01'
"""

# Execute the query and load the data into a DataFrame
sales_data = pd.read_sql_query(query_sales, conn)
print("Sales Data Extracted:\n", sales_data.head())

In [ ]:
# Aggregating and summarizing sales data

query_summary = """
SELECT 
    s.Date, 
    SUM(s.Sales) AS Total_Sales, 
    SUM(i.Sales) AS Total_International_Sales
FROM 
    sale_report s
JOIN 
    international_sale_report i ON s.Date = i.Date
GROUP BY 
    s.Date
HAVING 
    s.Date >= '2022-01-01'
"""

# Execute the query and load the summary data into a DataFrame
sales_summary = pd.read_sql_query(query_summary, conn)
print("Sales Summary Data:\n", sales_summary.head())


In [ ]:
# Creating a View for Repeated Use

create_view_query = """
CREATE VIEW sales_summary AS
SELECT 
    s.Date, 
    SUM(s.Sales) AS Total_Sales, 
    SUM(i.Sales) AS Total_International_Sales
FROM 
    sale_report s
JOIN 
    international_sale_report i ON s.Date = i.Date
GROUP BY 
    s.Date
HAVING 
    s.Date >= '2022-01-01'
"""

# Execute the create view query
conn.execute(create_view_query)

# Using the created view
query_view = "SELECT * FROM sales_summary"
sales_summary = pd.read_sql_query(query_view, conn)
print("Sales Summary Data from View:\n", sales_summary.head())


In [ ]:
# Data Loading and Visualization with Python

import matplotlib.pyplot as plt

# Convert Date column to datetime
sales_summary['Date'] = pd.to_datetime(sales_summary['Date'])

# Visualizing the sales data
plt.figure(figsize=(14, 7))

# Plot Total Sales
plt.plot(sales_summary['Date'], sales_summary['Total_Sales'], label='Total Sales', color='blue')

# Plot Total International Sales
plt.plot(sales_summary['Date'], sales_summary['Total_International_Sales'], label='Total International Sales', color='orange')

plt.xlabel('Date')
plt.ylabel('Sales Amount')
plt.title('Total Sales vs Total International Sales Over Time')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Closing the connection
conn.close()
